In [ ]:
import girder_client 
import json
import jsonschema
from jsonschema import validate

%load_ext autoreload
%autoreload 2
import adrcSchemaHelpers as hlprs


gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
_ = gc.authenticate(interactive=True)

In [ ]:

# ### Additional properties https://support.riverbed.com/apis/steelscript/reschema/jsonschema.html
with open("adrcNpSchema.json","r") as fp:
    dsaNpSchema = json.load(fp)

errDir = ['__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', '__weakref__',
          '_contents', '_matches_type', '_set', '_type_checker', '_word_for_instance_in_error_message', '_word_for_schema_in_error_message',
          'absolute_path', 'absolute_schema_path', 'args', 'cause', 'context', 'create_from', 'instance', 'json_path', 'message', 'parent', 'path', 'relative_path', 'relative_schema_path', 'schema', 'schema_path', 'validator', 'validator_value', 'with_traceback'] 

def validateNPJson(jsonData):
    try:
        validate(instance=jsonData, schema=dsaNpSchema)
    except jsonschema.exceptions.ValidationError as err:
#         print("START\n", err._contents(), "\nEND")
        return (False, err._contents())
    return (True,)

# for i in itemSet:
#     if 'meta' in i:
#         if 'npSchema' in i['meta']:
#             npSchema = i['meta']['npSchema']
#             isvalid = validateNPJson(npSchema)
#             if not isvalid:
#                 print(npSchema)

In [ ]:
yearsToScan = ['2020'] ### Hard coding some years so I can focus on a smaller subset

validated = 0

for f in gc.listFolder('638e2da11f75016b81fda12f',parentFolderType='collection'):
    if f['name'] in yearsToScan:
        print("YEAR=============",f['name'],"=================")
        ## GEt the subfolders..
        for sf in gc.listFolder(f['_id']):
            #http://candygram.neurology.emory.edu:8080/api/v1#!/resource/resource_getResourceItems_id_items
            itemSet = list(gc.getResource("resource/%s/items?type=folder&limit=10" % sf['_id']))
            itemSet1 = [item for item in gc.listResource("resource/%s/items?type=folder" % sf['_id'])]
            
            print(itemSet,"\n", itemSet1)
    #         hlprs.scanMetadata(gc,sf,updateGirder=True)
            ## This is where we can modify the scanMetadata file .. if we bundle in the validateNPJson function
            ## then we can ONLY update npSchema with valid data, and if the data is NOT valid we probably keep the data
            ## but stick it on like  rawSlideMetadata or whatever key you decide you like
            for i in itemSet:
                if 'meta' in i:
                    isValid = validateNPJson(i['meta'])
                    
                    if not isValid[0]:
                        
                        errorDict = isValid[1]
                        message, problemVal, instance = errorDict["message"], errorDict["path"][-1], errorDict["instance"]
                        
                        instance = instance if problemVal != "stainID" else instance.replace("~", "")
                        
                        i['meta']["npSchema"][problemVal] = message
                        gc.addMetadataToItem(i['_id'],i['meta'])
                        gc.addMetadataToItem(i['_id'],{'npWorking': {problemVal: instance}})
                        
                        
#                         condition = 'npSchema' in i['meta'] and "stainID" in i['meta']["npSchema"] and i['meta']["npSchema"]["stainID"] is not None and "~" in i['meta']["npSchema"]["stainID"]
                    
#                         if condition:
#                             stain = i['meta']["npSchema"]["stainID"]
#                             i['meta']["npSchema"]["stainID"] = "Stain ID Not Valid"
#                             gc.addMetadataToItem(i['_id'],i['meta'])
#                             gc.addMetadataToItem(i['_id'],{'npWorking': {"stainID": stain.replace("~", "")}})
                        
#                     if 'npWorking' in i['meta'] and not i['meta']['npWorking']:
#                         gc.addMetadataToItem(i['_id'],{'npWorking': None})


In [ ]:

validate(t,dsaNpSchema)